In [ ]:
!pip install streamlit
!pip install pyngrok


In [3]:
# Download haarcascade_frontalface_default.xml
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml


--2024-09-21 15:44:58--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml.1’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.06s   

2024-09-21 15:44:59 (15.1 MB/s) - ‘haarcascade_frontalface_default.xml.1’ saved [930127/930127]



In [4]:
from pyngrok import ngrok

# Replace 'YOUR_NGROK_AUTHTOKEN' with the actual authtoken you obtained
ngrok.set_auth_token("YOUR_NGROK_AUTHTOKEN")


In [13]:
%%writefile app.py
import streamlit as st
import os
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import datetime

# Function to ensure directories exist
def assure_path_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Function to get images and labels for training
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')]
    faces = []
    Ids = []
    for imagePath in imagePaths:
        pilImage = Image.open(imagePath).convert('L')  # grayscale
        imageNp = np.array(pilImage, 'uint8')
        # Assuming the image filename format: name.ID.sampleNum.jpg
        try:
            ID = int(os.path.split(imagePath)[-1].split(".")[1])
            faces.append(imageNp)
            Ids.append(ID)
        except:
            continue  # skip files that don't match the naming convention
    return faces, Ids

# Initialize session state for captured images
if 'captured_images' not in st.session_state:
    st.session_state.captured_images = []

# Initialize session state for attendance records
if 'attendance_records' not in st.session_state:
    st.session_state.attendance_records = []

st.title("FacePulse AI: Facial Attendance System")

menu = ["Register", "Train", "Attendance", "About"]
choice = st.sidebar.selectbox("Menu", menu)

if choice == "Register":
    st.subheader("Register New User")
    with st.form("register_form"):
        Id = st.text_input("Enter ID")
        name = st.text_input("Enter Name")
        submit = st.form_submit_button("Start Registration")

    if submit:
        if not Id or not name:
            st.error("Please enter both ID and Name.")
        elif not name.replace(' ', '').isalpha():
            st.error("Name should contain only alphabets and spaces.")
        else:
            st.success(f"Registration started for {name} with ID {Id}.")
            st.write("Please capture images by clicking the 'Capture Image' button below.")

            # Capture Images
            if st.button("Capture Image"):
                image = st.camera_input("Take a picture")
                if image:
                    img = Image.open(image)
                    img_np = np.array(img)
                    st.image(img, caption='Captured Image', use_column_width=True)
                    # Convert RGB to BGR for OpenCV
                    img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
                    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
                    faces = cv2.CascadeClassifier('haarcascade_frontalface_default.xml').detectMultiScale(gray, 1.3, 5)
                    for (x, y, w, h) in faces:
                        face_img = gray[y:y+h, x:x+w]
                        st.session_state.captured_images.append(face_img)
                        st.success(f"Image {len(st.session_state.captured_images)} captured.")
                        break  # Capture one face per image

            # Display number of captured images
            st.write(f"Total Images Captured: {len(st.session_state.captured_images)}")
            st.write("Capture at least 10 images for effective training.")

            # Save Images
            if st.button("Save Images") and len(st.session_state.captured_images) >= 10:
                assure_path_exists("TrainingImage")
                serial = 0
                # Check existing CSV to get the next serial number
                if os.path.isfile("StudentDetails/StudentDetails.csv"):
                    df_existing = pd.read_csv("StudentDetails/StudentDetails.csv")
                    serial = len(df_existing) + 1
                else:
                    columns = ['SERIAL NO.', 'ID', 'NAME']
                    df_existing = pd.DataFrame(columns=columns)
                    serial = 1

                # Save each captured image
                for idx, face in enumerate(st.session_state.captured_images):
                    img_path = f"TrainingImage/{name}.{Id}.{serial}.{idx+1}.jpg"
                    cv2.imwrite(img_path, face)

                # Save user details to CSV
                new_entry = {'SERIAL NO.': serial, 'ID': Id, 'NAME': name}
                df_existing = df_existing.append(new_entry, ignore_index=True)
                df_existing.to_csv("StudentDetails/StudentDetails.csv", index=False)

                st.success(f"Images and profile saved for {name} with ID {Id}.")

                # Clear captured images from session state
                st.session_state.captured_images = []
            elif st.button("Save Images") and len(st.session_state.captured_images) < 10:
                st.warning("Please capture at least 10 images before saving.")

elif choice == "Train":
    st.subheader("Train the Model")
    if st.button("Train Images"):
        if not os.path.exists("TrainingImage"):
            st.error("No training images found. Please register users first.")
        else:
            faces, Ids = getImagesAndLabels("TrainingImage")
            if len(faces) == 0:
                st.error("No valid training images found.")
            else:
                recognizer = cv2.face.LBPHFaceRecognizer_create()
                recognizer.train(faces, np.array(Ids))
                recognizer.save("TrainingImageLabel/Trainer.yml")
                st.success("Model trained and saved successfully.")

elif choice == "Attendance":
    st.subheader("Take Attendance")
    if st.button("Start Attendance"):
        if not os.path.isfile("TrainingImageLabel/Trainer.yml"):
            st.error("Model not found. Please train the model first.")
        elif not os.path.isfile("StudentDetails/StudentDetails.csv"):
            st.error("Student details not found. Please register users first.")
        else:
            # Load the trained model
            recognizer = cv2.face.LBPHFaceRecognizer_create()
            recognizer.read("TrainingImageLabel/Trainer.yml")

            # Load student details
            df = pd.read_csv("StudentDetails/StudentDetails.csv")

            # Capture attendance via webcam
            st.write("Starting webcam for attendance tracking...")
            cap = cv2.VideoCapture(0)
            stframe = st.empty()
            attendance = []

            while True:
                ret, frame = cap.read()
                if not ret:
                    st.error("Failed to access the webcam.")
                    break
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = cv2.CascadeClassifier('haarcascade_frontalface_default.xml').detectMultiScale(gray, 1.3, 5)

                for (x, y, w, h) in faces:
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                    serial, conf = recognizer.predict(gray[y:y+h, x:x+w])
                    if conf < 50:
                        name = df.loc[df['SERIAL NO.'] == serial]['NAME'].values
                        ID = df.loc[df['SERIAL NO.'] == serial]['ID'].values
                        name = name[0] if len(name) > 0 else "Unknown"
                        ID = ID[0] if len(ID) > 0 else "Unknown"
                        cv2.putText(frame, str(name), (x, y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

                        # Mark attendance
                        ts = datetime.datetime.now()
                        date = ts.strftime('%d-%m-%Y')
                        timeStamp = ts.strftime('%H:%M:%S')
                        attendance_record = {'ID': ID, 'Name': name, 'Date': date, 'Time': timeStamp}
                        if attendance_record not in attendance:
                            attendance.append(attendance_record)
                            # Save to CSV
                            attendance_file = f"Attendance/Attendance_{date}.csv"
                            if not os.path.isfile(attendance_file):
                                df_attendance = pd.DataFrame(columns=['ID', 'Name', 'Date', 'Time'])
                            else:
                                df_attendance = pd.read_csv(attendance_file)
                            df_attendance = df_attendance.append(attendance_record, ignore_index=True)
                            df_attendance.to_csv(attendance_file, index=False)
                            st.success(f"Attendance marked for {name} at {timeStamp}")
                    else:
                        cv2.putText(frame, "Unknown", (x, y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                # Display the frame in Streamlit
                stframe.image(frame, channels='BGR')

                # To stop the loop, press 'q' in the Streamlit interface (Not directly possible, but you can limit the duration)
                # For demonstration, let's capture for a fixed number of frames or duration
                # Here, we'll break the loop after 100 frames
                if len(attendance) >= 100:
                    break

            cap.release()
            cv2.destroyAllWindows()
            st.success("Attendance tracking completed.")

elif choice == "About":
    st.subheader("About")
    st.text("FacePulse AI is a facial recognition-based attendance monitoring system.")
    st.text("Developed using Streamlit, OpenCV, and Python.")


Overwriting app.py


In [ ]:
!ngrok http 8501

In [15]:
# Open a tunnel to the Streamlit port 8501
public_url = ngrok.connect(8501)
print(f"Streamlit app is running at: {public_url}")


Streamlit app is running at: NgrokTunnel: "https://8a25-34-29-189-148.ngrok-free.app" -> "http://localhost:8501"


In [16]:
import os
import time

# Run the Streamlit app in the background
get_ipython().system_raw('streamlit run app.py &')

# Optional: Wait for the app to initialize
time.sleep(5)
